In [1]:
import ast
import re
from tqdm import tqdm
import json

In [2]:

# Specify the path to your file
input_filename = "D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/segment_entier/contained_all_segments.txt"
output_filename = "D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/segment_entier/filtered_segments.txt"
output_filename_20 = "D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/segment_entier/filtered_segments_20.txt"

In [5]:
import time

def print_progress_bar(iteration, total, prefix='', suffix='', length=50, fill='█'):
    percent = ("{0:.1f}").format(100 * (iteration / float(total)))
    filled_length = int(length * iteration // total)
    bar = fill * filled_length + '-' * (length - filled_length)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end='\r')
    # Print New Line on Complete
    if iteration == total:
        print()

In [ ]:
# Open the input file for reading
with open(input_filename, 'r') as input_file:
    # Open the output file for writing
    with open(output_filename, 'w') as output_file:
        for line_num, line in enumerate(input_file, 1):
            # Extract relevant information from the line
            segment_info = line.strip().split(', ')
            print(segment_info)
            segment_id = int(segment_info[0].split(': ')[1])
            print(segment_id)

            # Extract the pixel coordinates using regular expression
            pixels_str = re.search(r'Pixels: \[\[(.*?)\]\]', line)
            print(pixels_str)
            if pixels_str:
                pixels_str = pixels_str.group(1)
                pixels = [list(map(int, coord.split(', '))) for coord in pixels_str.split('], [')]

                # Check if the segment is in the upper-left corner
                if all(0 <= x <= 2256 and 0 <= y <= 2591 for x, y in pixels):
                    # Write the line to the new file
                    output_file.write(line)

            # Update Progress Bar
            #print_progress_bar(line_num, total_lines, prefix='Progress:', suffix='Complete', length=50)

print(f"\nFiltered segments saved to {output_filename}")

In [7]:
import pandas as pd

# Load the filtered segments into a DataFrame
filtered_df = pd.read_csv(output_filename_20, sep=',', header=None, low_memory=False, on_bad_lines='skip')
# Drop columns 0, 1, and 2
filtered_df = filtered_df.drop(columns=[0, 1, 2])
# Concatenate the remaining columns into one
filtered_df['Concatenated Pixels'] = filtered_df.apply(lambda row: ''.join(map(str, row)), axis=1)
filtered_df = filtered_df[['Concatenated Pixels']]

In [8]:
# Remove the 'Pixels: ' prefix and convert the remaining string to a list
filtered_df['Pixels'] = filtered_df['Concatenated Pixels'].str.replace('Pixels: ', '')
filtered_df = filtered_df[['Pixels']]

In [9]:
def convert_to_tuple_list(pixel_str):
    if isinstance(pixel_str, float) and np.isnan(pixel_str):
        return []
    pixel_str = pixel_str.replace('[', '').replace(']', '').replace('nan', '')
    pixel_list = pixel_str.split()
    return [(int(pixel_list[i]), int(pixel_list[i + 1])) for i in range(0, len(pixel_list), 2) if 'nan' not in pixel_list[i]]

filtered_df['Pixels'] = filtered_df['Pixels'].apply(convert_to_tuple_list)

In [10]:
max_x = max_y = 0

for pixel_list in filtered_df['Pixels']:
    for pixel in pixel_list:
        x, y = pixel
        max_x = max(max_x, x)
        max_y = max(max_y, y)

print(f"Max X: {max_x}, Max Y: {max_y}")

Max X: 225, Max Y: 259


In [57]:

filtered_df.head()

,Pixels
0,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0..."
1,"[(0, 5), (0, 6), (0, 7), (0, 8), (1, 5), (1, 6..."
2,"[(0, 9), (0, 10), (0, 11), (0, 12), (1, 9), (1..."
3,"[(0, 13), (0, 14), (0, 15), (0, 16), (1, 13), ..."
4,"[(0, 17), (0, 18), (0, 19), (1, 16), (1, 17), ..."
